# Collaborative Filtering 

협업필터링은 비슷한 성향을 가진 사용자는 비슷한 아이템을 선호 할 것이라는 가정에서 출발한 알고리즘입니다. 사용자가 아이템에 대해서 투표한 평점 로그 데이터를 이용하며, 사용자나 아이템을 기준으로 평점의 유사성을 살피는 Memory-based CF와 평점 데이터를 통해 모델을 만들고 평점을 예측하는 Model-based CF이 있습니다.

## Memory-based CF(Collaborative Filtering)

평점 행렬을 기억하고 평점 예측을 위해 기억된 행렬 기반으로 유사도 계산을 진행하기 때문에 Memory-based CF라고 불립니다. 이 방법들은 평점 데이터가 충분히 있을 경우 간단하면서 좋은 성능을 나타내지만, 평점 데이터가 없는 새로운 유저 및 아이템 추가시 대응이 어렵다는 단점이 있습니다.

![matrix](https://cdn-images-1.medium.com/max/1600/1*NTd4NH0H1mrSr01Ppr58dg.png)

**User-based CF**
* 평점 행렬에서 사용자 평점 벡터 기준으로 유사한 사용자를 찾아서 이를 기반으로 원하는 아이템의 평점을 계산하는 방법
* 예) E 사용자의 TV 아이템 평점을 예측하고자 할때, E 사용자와 비슷한 아이템 평점 벡터를 가진 B, C 사용자의 TV 아이템 평점벡터를 기준으로 예측
  
**Item-based CF**
* 평점 행렬에서 아이템 평점 벡터 기준으로 유사한 아이템를 찾아서 이를 기반으로 원하는 사용지의 평점을 계산하는 방법
* 예) TV 아이템의 E 사용자 평점을 예측하고자 할대, TV 아이템과 비슷한 사용자 평점 벡터를 가진 게임패트 아이템의 사용자 평점벡터를 기준으로 예측


유사도 계산을 위해 surprise 패키지는 다음과 같은 유사도 기준을 제공합니다.
* 평균제곱차이 유사도 ('msd': Mean Squared Difference Similarity)
* 코사인 유사도 ('cosine': Cosine Similarity)
* 피어슨 유사도 ('psearson': Pearson Similarity)
* 피어슨-베이스라인 유사도 ('pearson_baseline': Pearson-Baseline Similarity)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import os, math, random
import numpy as np
import pandas as pd
from sklearn import datasets, preprocessing, model_selection, metrics
import surprise as sp

seed = 0
random.seed(seed)
np.random.seed(seed)

# 데이터
sp_data = sp.Dataset.load_builtin('ml-100k')
df_data = pd.DataFrame(sp_data.raw_ratings, columns=["user_id", "item_id", "rating", "timestamp"])
print("df_data.shape={}".format(df_data.shape))
print(df_data.dtypes)
print(df_data.head())
print(df_data.describe(include='all'))

df_train, df_test = model_selection.train_test_split(df_data, test_size=0.1)
print(df_data.shape, df_train.shape, df_test.shape)

# 전처리 
# A reader is still needed but only the rating_scale param is requiered.
reader = sp.Reader(rating_scale=(1, 5))
sp_data = sp.Dataset.load_from_df(df_train[['user_id', 'item_id', 'rating']], reader)
# surprise model.test 의 input shape => [(user_id, item_id, rating)]
sp_test = [(row['user_id'], row['item_id'], row['rating']) for i, row in df_test.iterrows()]

# 모델
models = [
    sp.KNNBasic(sim_options={'name' : 'msd'}), 
    sp.KNNBasic(sim_options={'name' : 'cosine'}),
    sp.KNNBasic(sim_options={'name' : 'pearson'}),
    sp.KNNBasic(sim_options={'name' : 'msd', 'user_based': False}),
    sp.KNNBasic(sim_options={'name' : 'cosine', 'user_based': False}),
    sp.KNNBasic(sim_options={'name' : 'pearson', 'user_based': False})    
]

for model in models:
    # 학습
    sp.model_selection.cross_validate(model, sp_data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
    
    # 평가
    sp_pred = model.test(sp_test)
    rmse = sp.accuracy.rmse(sp_pred, verbose=False)
    print("Test RMSE={}".format(rmse))

df_data.shape=(100000, 4)
user_id       object
item_id       object
rating       float64
timestamp     object
dtype: object
  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596
       user_id item_id         rating  timestamp
count   100000  100000  100000.000000     100000
unique     943    1682            NaN      49282
top        405      50            NaN  891033606
freq       737     583            NaN         12
mean       NaN     NaN       3.529860        NaN
std        NaN     NaN       1.125674        NaN
min        NaN     NaN       1.000000        NaN
25%        NaN     NaN       3.000000        NaN
50%        NaN     NaN       4.000000        NaN
75%        NaN     NaN       4.000000        NaN
max        NaN     NaN       5.000000        NaN
(100000, 4) (90000, 4) (10000, 4)
Computing the msd similarity mat

predict() 함수를 통해 한 데이터 셋에 대해서 예측 평점을 구할 수 있으며, Top-N 개의 비슷한 아이템이나, 사용자를 찾을 수 있습니다.

In [2]:
# 평가: by predict
tests = []
preds = []
for row in sp_test:
    tests.append(row[2])
    pred = model.predict(row[0], row[1], row[2])
    preds.append(pred.est)

rmse = math.sqrt(metrics.mean_squared_error(tests, preds))
print("Test RMSE={}".format(rmse))

Test RMSE=1.051538692275093


In [3]:
print("Top-N Similar Users")
user_model = models[0] # sp.KNNBasic(sim_options={'name' : 'msd'})
raw_uid = '22'
inner_uid = user_model.trainset.to_inner_uid(raw_uid)
raw_uid = user_model.trainset.to_raw_uid(inner_uid)
print("raw_uid:{} == inner_uid:{}".format(raw_uid, inner_uid))

top_inner_uids = user_model.get_neighbors(inner_uid, k=5)
print("top-5 inner_uids: {}".format(top_inner_uids))
top_raw_uids = [user_model.trainset.to_raw_uid(top_inner_uid) for top_inner_uid in top_inner_uids]
print("top-5 raw_uids: {}".format(top_raw_uids))

print("\nTop-N Similar Items")
item_model = models[3] # sp.KNNBasic(sim_options={'name' : 'msd', 'user_based': False})
raw_iid = '377'
inner_iid = item_model.trainset.to_inner_iid(raw_iid)
raw_iid = item_model.trainset.to_raw_iid(inner_iid)
print("raw_iid:{} == inner_iid:{}".format(raw_iid, inner_iid))

top_inner_iids = item_model.get_neighbors(inner_iid, k=5)
print("top-5 inner_iids: {}".format(top_inner_iids))
top_raw_iids = [item_model.trainset.to_raw_iid(top_inner_iid) for top_inner_iid in top_inner_iids]
print("top-5 raw_iids: {}".format(top_raw_iids))

Top-N Similar Users
raw_uid:22 == inner_uid:493
top-5 inner_uids: [3, 32, 85, 100, 174]
top-5 raw_uids: ['803', '703', '494', '787', '802']

Top-N Similar Items
raw_iid:377 == inner_iid:610
top-5 inner_iids: [1, 15, 30, 47, 56]
top-5 raw_iids: ['597', '402', '356', '365', '406']


## Model-based CF

평점 데이터를 통해 모델을 만드는 방법은 다양한 접근이 가능하지만, 그 중 행렬의 연산을 이용하여 특징벡터를 추출하여 사용하는 Matrix Factorization을 많이 사용합니다. Matrix Factorization 문제에 대한 해를 찾는 방법은 여러가지가 있지만 그 중에서도 일반적으로 SVD(Singular Value Decomposition)방법을 사용합니다.

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import os, math
import numpy as np
import pandas as pd
from sklearn import datasets, preprocessing, model_selection, metrics
import surprise as sp

# 데이터
sp_data = sp.Dataset.load_builtin('ml-100k')
df_data = pd.DataFrame(sp_data.raw_ratings, columns=["user_id", "item_id", "rating", "timestamp"])
print("df_data.shape={}".format(df_data.shape))
print(df_data.dtypes)
print(df_data.head())
print(df_data.describe(include='all'))

df_train, df_test = model_selection.train_test_split(df_data, test_size=0.1)
print(df_data.shape, df_train.shape, df_test.shape)

# 전처리 
# A reader is still needed but only the rating_scale param is requiered.
reader = sp.Reader(rating_scale=(1, 5))
sp_data = sp.Dataset.load_from_df(df_train[['user_id', 'item_id', 'rating']], reader)
# surprise model.test 의 input shape => [(user_id, item_id, rating)]
sp_test = [(row['user_id'], row['item_id'], row['rating']) for i, row in df_test.iterrows()]

# 모델
models = [
    sp.SVD(n_factors=20),    
]

for model in models:
    # 학습
    sp.model_selection.cross_validate(model, sp_data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
    
    # 평가
    sp_pred = model.test(sp_test)
    rmse = sp.accuracy.rmse(sp_pred, verbose=False)
    print("Test RMSE={}".format(rmse))

df_data.shape=(100000, 4)
user_id       object
item_id       object
rating       float64
timestamp     object
dtype: object
  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596
       user_id item_id         rating  timestamp
count   100000  100000  100000.000000     100000
unique     943    1682            NaN      49282
top        405      50            NaN  891033606
freq       737     583            NaN         12
mean       NaN     NaN       3.529860        NaN
std        NaN     NaN       1.125674        NaN
min        NaN     NaN       1.000000        NaN
25%        NaN     NaN       3.000000        NaN
50%        NaN     NaN       4.000000        NaN
75%        NaN     NaN       4.000000        NaN
max        NaN     NaN       5.000000        NaN
(100000, 4) (90000, 4) (10000, 4)
Evaluating RMSE, MAE of algorith

predict() 함수를 통해 한 데이터 셋에 대해서 예측 평점을 구할 수 있습니다.

In [5]:
# 평가: by predict
tests = []
preds = []
for row in sp_test:
    tests.append(row[2])
    pred = model.predict(row[0], row[1], row[2])
    preds.append(pred.est)

rmse = math.sqrt(metrics.mean_squared_error(tests, preds))
print("Test RMSE={}".format(rmse))

Test RMSE=0.9465768404143768
